In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Carregando os dados do arquivo Excel
file_path = 'latencia2.xlsx'  # Substitua pelo caminho do seu arquivo
df = pd.read_excel(file_path)

# Convertendo a coluna "latencies" de string para uma lista de números
df['latencies'] = df['latencies'].apply(lambda x: [int(n) for n in x.split(',')])

# Rotulando os dados: tempos de resposta acima do 60º percentil como anômalos (1), outros como normais (0)
percentil_60 = np.percentile(df['latencies'].explode(), 60)
df['labels'] = df['latencies'].apply(lambda x: [1 if i > percentil_60 else 0 for i in x])

# Preparando os dados de treinamento e teste
X = []
y = []
for _, row in df.iterrows():
    for latency, label in zip(row['latencies'], row['labels']):
        X.append([latency])  # Features são os tempos de resposta
        y.append(label)      # Labels são os indicadores de anomalia

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inicializando variáveis para monitorar a melhor acurácia e o melhor modelo
best_accuracy = 0
best_model = None

# Treinando o modelo com um número crescente de árvores e salvando o melhor modelo
for n_trees in range(1, 101):  # de 1 até 100 árvores
    clf = RandomForestClassifier(n_estimators=n_trees, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Época {n_trees}: Acurácia = {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = clf

# Salvando o melhor modelo
model_filename = 'best_random_forest_model.pkl'
joblib.dump(best_model, model_filename)

print(f"Melhor modelo salvo com acurácia de {best_accuracy:.4f}")


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Output, Input
import plotly.graph_objs as go
import joblib
import random

# Carregar o modelo de previsão de anomalias
model = joblib.load('best_random_forest_model.pkl')

# Inicializar aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    dcc.Graph(id='live-graph', animate=True),
    dcc.Interval(
        id='graph-update',
        interval=1000,  # em milissegundos, atualiza a cada 1 segundo
        n_intervals=0
    ),
])

# Callback para atualizar o gráfico automaticamente
@app.callback(
    Output('live-graph', 'figure'),
    [Input('graph-update', 'n_intervals')]
)
def update_graph(n_intervals):
    # Gerar latência aleatoriamente até 1.000.000
    random_latency = random.randint(0, 1000000)

    # Prever anomalia
    prediction = model.predict([[random_latency]])

    # Definir cor da barra com base na previsão
    color = 'red' if prediction[0] == 1 else 'blue'

    # Criar gráfico
    data = go.Bar(
            x=[n_intervals],
            y=[random_latency],
            marker_color=color
    )

    # Definir o layout do gráfico, ajustando a formatação do eixo y
    layout = go.Layout(
        xaxis=dict(range=[max(0, n_intervals-50), n_intervals]),
        yaxis=dict(
            range=[0, max(500000, random_latency)],
            title='Latência (ms)',
            tickformat=','
        )
    )

    return {'data': [data], 'layout': layout}

# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)
